In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:

results_h = 'results_h.jsonl'
results_t = 'results_t.jsonl'
best_model_f = 'best_model_f.jsonl'


In [ ]:
!mkdir ckpts
!mkdir reports
!mkdir results

In [ ]:
!pip install GPUtil pytorch_pretrained_bert transformers
!pip install barbar
!pip uninstall emoji
!pip install emoji==1.7
!pip install emojis
!pip install --upgrade urllib3

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.8/123.8 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.8/135.8 kB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 26.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 54.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 86.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.4/143.4 kB 20.3 MB/s eta 0:00:00
  Created wheel for GPUtil: filename=GPUtil-1.4.0-py3-none-any.whl size=7392 sha256=27a032cfce5d15f8e2a7b803e486493bbb8b4b47e421c1e2999b8e69834a81c6
  Stored in directory: /root/.cache/pip/wheels/a9/8a/bd/81082387151853ab8b6b3ef33426e98

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import json

from sklearn.metrics import accuracy_score, f1_score, classification_report
import torch
from torch.utils.data import Dataset
from transformers import AutoTokenizer, AutoModel
import torch.nn as nn
import torch.nn.functional as F
import argparse
import numpy as np
import pandas as pd
import random
from transformers import AdamW, get_linear_schedule_with_warmup
from barbar import Bar
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import os
import re
from emoji import UNICODE_EMOJI
import sys
import emojis
import numpy
import shutil

from statistics import mean
import math
from torch.autograd import Variable

In [ ]:
class GRUCell(nn.Module):
    def __init__(self, input_size, hidden_size, bias=True):
        super(GRUCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.bias = bias

        self.x2h = nn.Linear(input_size, 3 * hidden_size, bias=bias)
        self.h2h = nn.Linear(hidden_size, 3 * hidden_size, bias=bias)

        self.reset_parameters()


    def reset_parameters(self):
        std = 1.0 / np.sqrt(self.hidden_size)
        for w in self.parameters():
            w.data.uniform_(-std, std)

    def forward(self, input, hx=None):

        # Inputs:
        #       input: of shape (batch_size, input_size)
        #       hx: of shape (batch_size, hidden_size)
        # Output:
        #       hy: of shape (batch_size, hidden_size)

        if hx is None:
            hx = Variable(input.new_zeros(input.size(0), self.hidden_size))

        x_t = self.x2h(input)
        h_t = self.h2h(hx)


        x_reset, x_upd, x_new = x_t.chunk(3, 1)
        h_reset, h_upd, h_new = h_t.chunk(3, 1)

        reset_gate = torch.sigmoid(x_reset + h_reset)
        update_gate = torch.sigmoid(x_upd + h_upd)
        new_gate = torch.tanh(x_new + (reset_gate * h_new))

        hy = update_gate * hx + (1 - update_gate) * new_gate

        return hy


class GRU(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, bias, output_size):
        super(GRU, self).__init__()

        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.bias = bias
        self.output_size = output_size

        self.rnn_cell_list = nn.ModuleList()

        self.rnn_cell_list.append(GRUCell(self.input_size,
                                          self.hidden_size,
                                          self.bias))
        for l in range(1, self.num_layers):
            self.rnn_cell_list.append(GRUCell(self.hidden_size,
                                              self.hidden_size,
                                              self.bias))
        self.fc = nn.Linear(self.hidden_size, self.output_size)


    def forward(self, input, hx=None):

        # Input of shape (batch_size, seqence length, input_size)
        #
        # Output of shape (batch_size, output_size)

        if hx is None:
            if torch.cuda.is_available():
                h0 = Variable(torch.zeros(self.num_layers, input.size(0), self.hidden_size).cuda())
            else:
                h0 = Variable(torch.zeros(self.num_layers, input.size(0), self.hidden_size))

        else:
             h0 = hx

        outs = []

        hidden = list()
        for layer in range(self.num_layers):
            hidden.append(h0[layer, :, :])

        for t in range(input.size(1)):

            for layer in range(self.num_layers):

                if layer == 0:
                    hidden_l = self.rnn_cell_list[layer](input[:, t, :], hidden[layer])
                else:
                    hidden_l = self.rnn_cell_list[layer](hidden[layer - 1],hidden[layer])
                hidden[layer] = hidden_l

                hidden[layer] = hidden_l

            outs.append(hidden_l)

        # Take only last time step. Modify for seq to seq
        out = outs[-1].squeeze()

        out = self.fc(out)

        return out

# Dataset


In [ ]:
class TrainDataset(Dataset):
    def __init__(self, df, pretraine_path='xlm-roberta-base', max_length=128):
        self.df = df
        self.max_length = max_length

        self.tokenizer = AutoTokenizer.from_pretrained(pretraine_path)

    def __getitem__(self, index):
        text = self.df.iloc[index]['tweet']
        label = self.df.iloc[index]["sarcastic"]

        encoded_input = self.tokenizer(
                text,
                max_length = self.max_length,
                padding='max_length',
                truncation=True,
                return_tensors="pt",
            )

        input_ids = encoded_input["input_ids"]
        attention_mask = encoded_input["attention_mask"] if "attention_mask" in encoded_input else None

        data_input = {
            "input_ids":input_ids.flatten(),
            "attention_mask": attention_mask.flatten()
        }

        label_input ={
            "sarcasm": torch.tensor(label, dtype=torch.float),
        }

        return data_input, label_input

    def __len__(self):
        return self.df.shape[0]

In [ ]:
def label_rule(mis_preds, cat_preds):
    for i in range(len(mis_preds)):
        if mis_preds[i] == 0:
            cat_preds[i] = 0
    return cat_preds


def accuracy(preds, y):
    all_output = preds.float().cpu()
    all_label = y.float().cpu()
    _, predict = torch.max(all_output, 1)
    acc = accuracy_score(all_label.numpy(), torch.squeeze(predict).float().numpy())
    return acc

def calc_accuracy(preds,y):
    predict = torch.argmax(preds, dim=1)
    accuracy = torch.sum(predict == y.squeeze()).float().item()
    return accuracy / float(preds.size()[0])

def calc_f1_sarcasm(preds,y):

    predict = torch.argmax(preds, dim=1)
    f1_sarcastic = f1_score(y, predict, average='binary', pos_label=1)
    return f1_sarcastic

def calc_f1_score(preds,y):
    predict = torch.argmax(preds, dim=1)
    f1_score_macro = f1_score(y, predict, average='macro')
    return f1_score_macro

def binary_accuracy2(preds, y):
    # round predictions to the closest integer
    rounded_preds = torch.round(preds).squeeze()

    correct = (rounded_preds == y).float()
    acc = correct.sum() / (y.size(0))
    return acc

def binary_accuracy(preds, y):
    # round predictions to the closest integer
    rounded_preds = torch.round(preds).squeeze()

    correct = (rounded_preds == y).float()
    acc = correct.sum() / (y.size(0) * y.size(1))
    return acc

# preprocessing

In [ ]:

dic = {
      "egypt": 'المصرية',
	  "nile": 'المصرية',
	  "msa": "اللغة العربية الفصحى",
	  "magreb": "المغربية",
	  "gulf": "الخليجية",
	  "levant": "الشامية"
}

def is_emoji(s):
    return s in UNICODE_EMOJI

# add space near your emoji
def add_space(text):
    return ''.join(' ' + char if is_emoji(char) else char for char in text).strip()

def preprocess(text):
    sent = add_space(text)
    sent = re.sub(r'(?:@[\w_]+)', "user", sent)
    sent = re.sub(r'http[s]?://(?:[a-z]|[0-9]|[$-_@.&amp;+]|[!*\(\),]|(?:%[0-9a-f][0-9a-f]))+', "url", sent)
    sent = sent.replace('_', ' ')
    sent = sent.replace('#', ' ')
    return sent

def prepare_text(df, col):
    if col == 'tweet':
        df['dialect'] = df['dialect'].map(dic)
    for i in range(df.shape[0]):
        df.loc[i, col] = df.loc[i, 'dialect'] + ' [SEP] ' + df.loc[i, col]


    return df


In [ ]:
def loadTrainValData2(size=0.2, batchsize=16, num_worker=0, pretraine_path="marbert", seed=42, max_length=128):
    path = "/content/drive/MyDrive/iSarcasm/Datasets2/split_80_20/train.csv"
    data = pd.read_csv(path, encoding='utf-8')
    data = data[~data.tweet.isna()]
    data['tweet'] = data['tweet'].apply(lambda x: preprocess(x))
    data = prepare_text(data, col='tweet')

    rephrase_df = data[["rephrase", "dialect"]]
    rephrase_df = rephrase_df.dropna().reset_index(drop=True)
    rephrase_df['rephrase'] = rephrase_df['rephrase'].apply(lambda x: preprocess(x))
    rephrase_df = prepare_text(rephrase_df, col='rephrase')
    rephrase_df = rephrase_df[["rephrase"]]

    rephrase_df["sarcastic"] = 0
    rephrase_df.columns = ["tweet", "sarcastic"]
    data = data[["tweet", "sarcastic"]]
    data = data.sample(frac=1).reset_index(drop=True)
    df_train = data
    df_train = pd.concat([df_train, rephrase_df])
    df_train = df_train.sample(frac=1).reset_index(drop=True)

    path = "/content/drive/MyDrive/iSarcasm/Datasets2/split_80_20/val.csv"
    data = pd.read_csv(path, encoding='utf-8')
    data = data[~data.tweet.isna()]
    data['tweet'] = data['tweet'].apply(lambda x: preprocess(x))
    data = prepare_text(data, col='tweet')
    df_test = data
    DF_train = TrainDataset(df_train, pretraine_path, max_length)
    DF_test = TrainDataset(df_test, pretraine_path, max_length)

    DF_train_loader = DataLoader(dataset=DF_train, batch_size=batchsize, shuffle=True,
                                 num_workers=num_worker)
    DF_test_loader = DataLoader(dataset=DF_test, batch_size=batchsize, shuffle=False,
                                num_workers=num_worker)
    return DF_train_loader, DF_test_loader

In [ ]:
def loadTestData(batchsize=16, num_worker=2, pretraine_path="xlm-roberta-base", max_length=128):
    path = "/content/drive/MyDrive/iSarcasm/Datasets2/tweet/task_A_Ar_test.csv"
    data = pd.read_csv(path, encoding='utf-8')
    data['tweet'] = data['tweet'].apply(lambda x:preprocess(x))
    data = prepare_text(data, col='tweet')

    DF_test = TrainDataset(data, pretraine_path, max_length)

    DF_test_loader = DataLoader(dataset=DF_test, batch_size=batchsize, shuffle=False,
                                num_workers=num_worker)


    return DF_test_loader



# Focal Loss

In [ ]:
class FocalLoss_Ori(nn.Module):
    """
    This is a implementation of Focal Loss with smooth label cross entropy supported which is proposed in
    'Focal Loss for Dense Object Detection. (https://arxiv.org/abs/1708.02002)'
    Focal_Loss= -1*alpha*((1-pt)**gamma)*log(pt)
    Args:
        num_class: number of classes
        alpha: class balance factor
        gamma:
        ignore_index:
        reduction:
    """

    def __init__(self, num_class, alpha=None, gamma=2, ignore_index=None, reduction='mean'):
        super(FocalLoss_Ori, self).__init__()
        self.num_class = num_class
        self.gamma = gamma
        self.reduction = reduction
        self.smooth = 1e-4
        self.ignore_index = ignore_index
        self.alpha = alpha
        if alpha is None:
            self.alpha = torch.ones(num_class, )
        elif isinstance(alpha, (int, float)):
            self.alpha = torch.as_tensor([alpha] * num_class)
        elif isinstance(alpha, (list, np.ndarray)):
            self.alpha = torch.as_tensor(alpha)
        if self.alpha.shape[0] != num_class:
            raise RuntimeError('the length not equal to number of class')

    def forward(self, logit, target):
        # assert isinstance(self.alpha,torch.Tensor)\
        N, C = logit.shape[:2]
        alpha = self.alpha.to(logit.device)
        prob = F.softmax(logit, dim=1)
        if prob.dim() > 2:
            # N,C,d1,d2 -> N,C,m (m=d1*d2*...)
            prob = prob.view(N, C, -1)
            prob = prob.transpose(1, 2).contiguous()  # [N,C,d1*d2..] -> [N,d1*d2..,C]
            prob = prob.view(-1, prob.size(-1))  # [N,d1*d2..,C]-> [N*d1*d2..,C]
        ori_shp = target.shape
        target = target.view(-1, 1)  # [N,d1,d2,...]->[N*d1*d2*...,1]
        valid_mask = None
        if self.ignore_index is not None:
            valid_mask = target != self.ignore_index
            target = target * valid_mask

        # ----------memory saving way--------
        prob = prob.gather(1, target).view(-1) + self.smooth  # avoid nan
        logpt = torch.log(prob)
        # alpha_class = alpha.gather(0, target.view(-1))
        alpha_class = alpha[target.squeeze().long()]
        class_weight = -alpha_class * torch.pow(torch.sub(1.0, prob), self.gamma)
        loss = class_weight * logpt
        if valid_mask is not None:
            loss = loss * valid_mask.squeeze()

        if self.reduction == 'mean':
            loss = loss.mean()
            if valid_mask is not None:
                loss = loss.sum() / valid_mask.sum()

        elif self.reduction == 'none':
            loss = loss.view(ori_shp)
        return loss





# modeling 2

In [ ]:


def init_weights(m):
    classname = m.__class__.__name__
    if classname.find('Conv2d') != -1 or classname.find('ConvTranspose2d') != -1:
        nn.init.kaiming_uniform_(m.weight)
        nn.init.zeros_(m.bias)
    elif classname.find('BatchNorm') != -1:
        nn.init.normal_(m.weight, 1.0, 0.02)
        nn.init.zeros_(m.bias)
    elif classname.find('Linear') != -1:
        nn.init.xavier_normal_(m.weight)
        if m.bias is not None:
            nn.init.zeros_(m.bias)




class TransformerLayer(nn.Module):
    def __init__(self,both=True,
                pretrained_path='aubmindlab/bert-base-arabert'):
        super(TransformerLayer, self).__init__()

        self.both = both
        self.transformer = AutoModel.from_pretrained(pretrained_path, output_hidden_states=True)


    def forward(self, input_ids=None, attention_mask=None):
        outputs = self.transformer(
            input_ids=input_ids,
            attention_mask=attention_mask
        )
        # (output_last_layer, pooled_cls, (output_layers))
        # output[0] (8, seqlen=64, 768) cls [8, 768] ( 12 (8, seqlen=64, 768))

        return outputs

    def output_num(self):
        return self.transformer.config.hidden_size

class ATTClassifier(nn.Module):
    def __init__(self, in_feature, class_num=1, dropout_prob=0.2):
        super(ATTClassifier, self).__init__()
        self.model = GRU(input_size=in_feature, hidden_size=in_feature, num_layers=1, bias=True , output_size=in_feature)

        self.Classifier = nn.Sequential(
            nn.Linear(2 * in_feature, 512),
            nn.Dropout(dropout_prob),
            nn.ReLU(),
            nn.Linear(512, class_num)
        )

        self.apply(init_weights)

    def forward(self, x):

        mod = self.model(x[0]) #(X[0] (bs, seqlenght, embedD) att = \sum_i alpha_i x[0][i]
        xx = torch.cat([mod, x[1]], 1)

        out = self.Classifier(xx)
        return out


# train_sarcat

In [ ]:


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


def train(base_model, mt_classifier, iterator, optimizer, sar_criterion, scheduler):

    # set the model in eval phase
    base_model.train(True)
    mt_classifier.train(True)

    acc_sarcasm= 0
    f1_sarcasm =0
    f1_score_macro = 0
    loss_sarc= 0

    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])

    for data_input, label_input  in Bar(iterator):

        for k, v in data_input.items():
            data_input[k] = v.to(device)

        for k, v in label_input.items():
            label_input[k] = v.long().to(device)

        optimizer.zero_grad()


        #forward pass

        sarcasm_target = label_input['sarcasm']

        # forward pass

        output = base_model(**data_input)
        sarcasm_logits = mt_classifier(output)

        sarcasm_probs = torch.softmax(sarcasm_logits, dim=1)

        loss_sarcasm = sar_criterion(sarcasm_logits, sarcasm_target)
        loss_sarc += loss_sarcasm.item()
        # backpropage the loss and compute the gradients
        loss_sarcasm.backward()
        optimizer.step()
        scheduler.step()
        acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
        _, predicted_sarcasm = torch.max(sarcasm_probs, 1)
        all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
        all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())

    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)


    accuracies = { 'accuracy': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score": fscore_macro}
    losses = { 'loss': loss_sarc / len(iterator)}
    return accuracies, losses


In [ ]:
def evaluate(base_model, mt_classifier, iterator, sar_criterion):
    # initialize every epoch
    acc_sarcasm= 0
    f1_sarcasm =0
    f1_score_macro = 0
    loss_sarc= 0

    #all_sarcasm_outputs = []
    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])

    # set the model in eval phase
    base_model.eval()
    mt_classifier.eval()
    with torch.no_grad():
        for data_input, label_input in Bar(iterator):

            for k, v in data_input.items():
                data_input[k] = v.to(device)

            for k, v in label_input.items():
                label_input[k] = v.long().to(device)


            sarcasm_target = label_input['sarcasm']

            # forward pass

            output = base_model(**data_input)
            sarcasm_logits = mt_classifier(output)
            logits = sarcasm_logits[:,:2]

            sarcasm_probs = torch.softmax(logits, dim=1)
            # compute the loss
            loss_sarcasm = sar_criterion(logits, sarcasm_target)

            # compute the running accuracy and losses
            acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)
            loss_sarc += loss_sarcasm.item()

            _, predicted_sarcasm = torch.max(sarcasm_probs, 1)
            #all_sarcasm_outputs.extend(predicted_sarcasm.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
            #all_sarcasm_labels.extend(sarcasm_target.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())
    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)

    fscore_macro = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)


    report_sarcasm = classification_report(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs,digits=4)


    accuracies = { 'accuracy': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score": fscore_macro, 'report_sarcasm': report_sarcasm}
    losses = { 'loss': loss_sarc / len(iterator)}
    return accuracies, losses


In [ ]:
def predict(base_model, mt_classifier, iterator):
    # initialize every epoch
    acc_sarcasm= 0
    loss_sarc= 0

    f1_sarcasm =0
    f1_score_macro = 0

    #all_sarcasm_outputs = []
    all_sarcasm_outputs = np.array([])
    all_sarcasm_labels = np.array([])

    # set the model in eval phase
    base_model.eval()
    mt_classifier.eval()
    with torch.no_grad():
      for data_input, label_input in Bar(iterator):
            for k, v in data_input.items():
                data_input[k] = v.to(device)

            for k, v in label_input.items():
                label_input[k] = v.long().to(device)

            sarcasm_target = label_input['sarcasm']

            # forward pass

            output = base_model(**data_input)
            sarcasm_logits = mt_classifier(output)
            logits = sarcasm_logits[:,:2]
            sarcasm_probs = torch.softmax(logits, dim=1)
            # compute the loss
            acc_sarcasm += calc_accuracy(sarcasm_probs, sarcasm_target)

            _, predicted_sarcasm = torch.max(sarcasm_probs, 1)
            #all_sarcasm_outputs.extend(predicted_sarcasm.squeeze().int().cpu().numpy().tolist())
            all_sarcasm_outputs = np.append(all_sarcasm_outputs, predicted_sarcasm.squeeze().cpu().numpy())
            all_sarcasm_labels = np.append(all_sarcasm_labels, sarcasm_target.squeeze().cpu().numpy())

    all_sarcasm_outputs = all_sarcasm_outputs.reshape(-1)
    all_sarcasm_labels = all_sarcasm_labels.reshape(-1)
    fscore_macro = f1_score(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs, average='macro')
    fscore_sarcasm = f1_score(all_sarcasm_labels, all_sarcasm_outputs, average='binary', pos_label=1)


    report_sarcasm = classification_report(y_true=all_sarcasm_labels, y_pred=all_sarcasm_outputs,digits=4)


    accuracies = { 'accuracy': acc_sarcasm / len(iterator), 'f1_sarcastic': fscore_sarcasm, "f1_score": fscore_macro, 'report_sarcasm': report_sarcasm}
    return accuracies, all_sarcasm_outputs, all_sarcasm_labels


In [ ]:
def train_full(config, train_loader, stest_loader):
    lr_o = config['lr_mult'] * config['lr']
    lr = config['lr']
    criterion = config['loss']
    #Instanciate models
    base_model = TransformerLayer(pretrained_path=config['pretrained_path'], both=True).to(device)
    mtl_classifier = ATTClassifier(base_model.output_num(), class_num=2).to(device)
    cls = 'ATTClassifier'

    if criterion =='FL':
        sarc_criterion = FocalLoss_Ori(num_class=2, alpha=[1, 1], gamma=2).to(device)
    else:
        sarc_criterion = nn.CrossEntropyLoss().to(device)
    #

    params = [{'params':base_model.parameters(), 'lr':config['lr']}, {'params': mtl_classifier.parameters(), 'lr': config['lr']}]#, {'params':multi_task_loss.parameters(), 'lr': 0.0005}]
    optimizer = AdamW(params, lr=config["lr"])

    train_data_size = len(train_loader)
    steps_per_epoch = int(train_data_size / config['batch_size'])
    num_train_steps = len(train_loader) * config['epochs']
    warmup_steps = int(config['epochs'] * train_data_size * 0.1 / config['batch_size'])

    scheduler = get_linear_schedule_with_warmup(optimizer,  num_warmup_steps=warmup_steps, num_training_steps=num_train_steps)
    # Train model

    best_val_accuracy = 0
    best_val_f1_score = 0
    best_val_f1_sarcastic = 0

    best_val_metric = 0

    best_val_loss = float('+inf')

    best_report_sarcasm = None

    epo = 0

    for epoch in range(config['epochs']):
        # print("epoch {}".format(epoch + 1))

        train_accuracies, train_losses = train(base_model, mtl_classifier, train_loader, optimizer, sarc_criterion,scheduler)
        valid_accuracies, valid_losses = evaluate(base_model, mtl_classifier, valid_loader, sarc_criterion)
        val_loss = valid_losses['loss']
        total_val_metric = valid_accuracies['f1_score']
        if epoch == (config['epochs'] - 1):

            epo = epoch+1
            best_val_loss = val_loss
            best_val_metric = total_val_metric

            best_val_f1_score = valid_accuracies['f1_score']
            best_val_f1_sarcastic = valid_accuracies['f1_sarcastic']
            best_report_sarcasm = valid_accuracies['report_sarcasm']
            best_val_accuracy= valid_accuracies['accuracy']
            best_val_loss = valid_losses['loss']

            print("save model's checkpoint")
            torch.save(base_model.state_dict(), f"./ckpts/best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat.pth")
            torch.save(mtl_classifier.state_dict(), f"./ckpts/best_cls_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat.pth")

    return best_val_accuracy, best_val_f1_score, best_val_f1_sarcastic, best_val_loss, epo

In [ ]:
def eval_full(config, loader1):
    criterion = config['loss']
    base_model = TransformerLayer(pretrained_path=config['pretrained_path'], both=True).to(device)
    classifier = ATTClassifier(base_model.output_num(), class_num=2).to(device)
    base_model.load_state_dict(torch.load(f"./ckpts/best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat.pth"))
    classifier.load_state_dict(torch.load(f"./ckpts/best_cls_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat.pth"))
    base_model = base_model.to(device)
    classifier = classifier.to(device)
    df = pd.DataFrame()

    accuracies, all_outputs_pred, all_outputs_label = predict(base_model, classifier, loader1)
    return accuracies, all_outputs_pred, all_outputs_label



In [ ]:
def plot_cf(cf_matrix):
    ax = sns.heatmap(cf_matrix, annot=True, cmap='Blues',fmt="d")
    ax.set_title('Seaborn Confusion Matrix with labels\n\n');
    ax.set_xlabel('\nPredicted Values')
    ax.set_ylabel('Actual Values ');
    ## Ticket labels - List must be in alphabetical order
    ax.xaxis.set_ticklabels(['False','True'])
    ax.yaxis.set_ticklabels(['False','True'])
    ## Display the visualization of the Confusion Matrix.
    plt.show()

In [ ]:
ls

ckpts/  drive/  reports/  results/  sample_data/


In [ ]:
lr_arr = [1e-5,2e-5]
batch_arr = [16,32,36,64]
epoch_arr = [2,4]
loss_arr=['FL']
seed_arr=[298]

In [ ]:
!mkdir '/content/drive/MyDrive/iSarcasm/m_298'

mkdir: cannot create directory ‘/content/drive/MyDrive/iSarcasm/m_298’: File exists


In [ ]:

# lr_arr = [2e-5]
# batch_arr = [16]
# epoch_arr = [2]
# loss_arr=['FL']
# seed_arr = [7774, 6911, 5751, 3723, 988, 2645, 3703, 2340, 298, 5423, 4759, 9054, 5495, 3120, 536, 16826, 3407 ]

In [ ]:
best_model = ''
best_f1 = 0

In [ ]:
for lr_i in lr_arr:
  for batch_i in batch_arr:
    for epoch_i in epoch_arr:
      for loss_i in loss_arr:
        for seed_i in seed_arr:

            args = {}
            args['lr_mult'] = 1.0
            args['seed'] = seed_i
            args['num_worker'] = 4
            args['lang'] = 'ar'
            args['phase'] = 'train'
            args['lm_pretrained'] = 'marbert'

            args['lr'] =lr_i
            args['epochs'] = epoch_i
            args['batch_size'] = batch_i
            args['loss'] = loss_i
            config = {}
            config["max_length"] = 64
            config['args'] = args
            config["output_for_test"] = True
            config['epochs'] = args['epochs']
            config["class_num"] = 1
            config["lr"] = args['lr']
            config['lr_mult'] = args['lr_mult']
            config['batch_size'] = args['batch_size']
            config['lm'] = args['lm_pretrained']
            config['loss'] = args['loss']
            lang = args['lang']
            dosegmentation = False


            if args['lm_pretrained'] == 'marbert':
                config['pretrained_path'] = "UBC-NLP/MARBERT"


            RANDOM_SEED = config['args']['seed']

            random.seed(RANDOM_SEED)
            np.random.seed(RANDOM_SEED)
            torch.manual_seed(RANDOM_SEED)
            torch.cuda.manual_seed(RANDOM_SEED)
            torch.cuda.manual_seed_all(RANDOM_SEED)
            torch.backends.cudnn.deterministic = True
            torch.backends.cudnn.benchmark = False


            model_name = f"{config['args']['batch_size']}_{config['args']['loss']}_{config['args']['lr']}_{config['args']['epochs']}_{config['args']['seed']}"
            print('model_name', model_name)
            train_loader, valid_loader = loadTrainValData2(size = 0.2 ,batchsize=args['batch_size'], num_worker=0, pretraine_path=config['pretrained_path'], max_length=config['max_length'])
            best_val_accuracy, best_val_f1_score, best_val_f1_sarcastic, best_val_loss, best_epo =train_full(config, train_loader, valid_loader)
            if (best_val_f1_score > best_f1):
              best_model = model_name
              best_f1 = best_val_f1_score

            results = {'model_name' : f'{model_name}'}
            results['loss_fn']= config['args']['loss']
            results['batch_size']=config['args']['batch_size']
            results['lr']=config['args']['lr']
            results['f1_sarcastic'] = best_val_f1_sarcastic
            results['f1_score'] = best_val_f1_score
            results['epoch']=config['args']['epochs']
            results['seed']=config['args']['seed']


            with open(results_h, 'a+') as f:
                  f.write(json.dumps(results) + '\n')



            args['phase'] = 'predict'
            config['args'] = args

            test_loader = loadTestData(batchsize=args['batch_size'], num_worker=0, pretraine_path=config['pretrained_path'],  max_length=config['max_length'])
            test_accuracies, test_all_outputs, test_all_labels = eval_full(config, loader1=test_loader)

            results = {'model_name' : f'{model_name}'}
            results['loss_fn']= config['args']['loss']
            results['batch_size']=config['args']['batch_size']
            results['lr']=config['args']['lr']
            results['f1_sarcastic'] = test_accuracies['f1_sarcastic']
            results['f1_score'] = test_accuracies['f1_score']
            results['epoch']=config['args']['epochs']
            results['seed']=config['args']['seed']
            with open(results_t, 'a+') as f:
                  f.write(json.dumps(results) + '\n')
            base_file=f"best_basemodel_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat.pth"
            cls_file=f"best_cls_sarcasm_{config['batch_size']}_{config['loss']}_{config['lr']}_ml{config['max_length']}_{config['epochs']}_{config['args']['seed']}_sarcat.pth"

            shutil.copy(f"./ckpts/{base_file}", f"/content/drive/MyDrive/iSarcasm/m_298/{base_file}")
            shutil.copy(f"./ckpts/{cls_file}", f"/content/drive/MyDrive/iSarcasm/m_298/{cls_file}")

model_name 16_FL_1e-05_2_298


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


3077/3077: [===============================>] - ETA 0.4s
621/621: [===============================>] - ETA 0.1s
3077/3077: [===============================>] - ETA 0.2s
621/621: [===============================>] - ETA 0.1s
save model's checkpoint
1400/1400: [===============================>] - ETA 0.1s
model_name 16_FL_1e-05_4_298


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


3077/3077: [===============================>] - ETA 0.3s
621/621: [===============================>] - ETA 0.1s
3077/3077: [===============================>] - ETA 0.3s
621/621: [===============================>] - ETA 0.1s
3077/3077: [===============================>] - ETA 0.3s
621/621: [===============================>] - ETA 0.1s
3077/3077: [===============================>] - ETA 0.3s
621/621: [===============================>] - ETA 0.1s
save model's checkpoint
1400/1400: [===============================>] - ETA 0.1s
model_name 32_FL_1e-05_2_298


...

3077/3077: [===============================>] - ETA 0.5s
621/621: [==============================>.] - ETA 0.2s
3077/3077: [===============================>] - ETA 0.4s
621/621: [==============================>.] - ETA 0.2s
3077/3077: [===============================>] - ETA 0.5s
621/621: [==============================>.] - ETA 0.1s
3077/3077: [===============================>] - ETA 0.5s
621/621: [==============================>.] - ETA 0.2s
save model's checkpoint
1400/1400: [===============================>] - ETA 0.2s
model_name 64_FL_2e-05_2_298


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


3077/3077: [===============================>] - ETA 0.8s
621/621: [============================>...] - ETA 0.2s
3077/3077: [===============================>] - ETA 0.8s
621/621: [============================>...] - ETA 0.3s
save model's checkpoint
1400/1400: [==============================>.] - ETA 0.3s
model_name 64_FL_2e-05_4_298


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


3077/3077: [===============================>] - ETA 0.8s
621/621: [============================>...] - ETA 0.3s
3077/3077: [===============================>] - ETA 0.8s
621/621: [============================>...] - ETA 0.2s
3077/3077: [===============================>] - ETA 0.7s
621/621: [============================>...] - ETA 0.3s
3077/3077: [===============================>] - ETA 0.7s
621/621: [============================>...] - ETA 0.2s
save model's checkpoint
1400/1400: [==============================>.] - ETA 0.3s


In [ ]:
results_db_h = pd.read_json(results_h, lines=True)
print("results_db_h", results_db_h)

results_db_h            model_name loss_fn  batch_size       lr  f1_sarcastic  f1_score  \
0   16_FL_1e-05_2_298      FL          16  0.00001      0.732824  0.830698   
1   16_FL_1e-05_4_298      FL          16  0.00001      0.749035  0.841455   
2   32_FL_1e-05_2_298      FL          32  0.00001      0.722222  0.825758   
3   32_FL_1e-05_4_298      FL          32  0.00001      0.747082  0.840546   
4   36_FL_1e-05_2_298      FL          36  0.00001      0.709163  0.817750   
5   36_FL_1e-05_4_298      FL          36  0.00001      0.750000  0.841258   
6   64_FL_1e-05_2_298      FL          64  0.00001      0.696356  0.810490   
7   64_FL_1e-05_4_298      FL          64  0.00001      0.754864  0.845452   
8   16_FL_2e-05_2_298      FL          16  0.00002      0.733591  0.831699   
9   16_FL_2e-05_4_298      FL          16  0.00002      0.726562  0.827784   
10  32_FL_2e-05_2_298      FL          32  0.00002      0.723735  0.825827   
11  32_FL_2e-05_4_298      FL          32  0.00002 

In [ ]:
results_db_t = pd.read_json(results_t, lines=True)
print("results_db_t", results_db_t)

results_db_t            model_name loss_fn  batch_size       lr  f1_sarcastic  f1_score  \
0   16_FL_1e-05_2_298      FL          16  0.00001      0.702439  0.825696   
1   16_FL_1e-05_4_298      FL          16  0.00001      0.663529  0.801659   
2   32_FL_1e-05_2_298      FL          32  0.00001      0.676617  0.811203   
3   32_FL_1e-05_4_298      FL          32  0.00001      0.634259  0.783768   
4   36_FL_1e-05_2_298      FL          36  0.00001      0.677249  0.813439   
5   36_FL_1e-05_4_298      FL          36  0.00001      0.629550  0.777698   
6   64_FL_1e-05_2_298      FL          64  0.00001      0.670330  0.810534   
7   64_FL_1e-05_4_298      FL          64  0.00001      0.652681  0.794919   
8   16_FL_2e-05_2_298      FL          16  0.00002      0.666667  0.803426   
9   16_FL_2e-05_4_298      FL          16  0.00002      0.655963  0.796256   
10  32_FL_2e-05_2_298      FL          32  0.00002      0.644231  0.791075   
11  32_FL_2e-05_4_298      FL          32  0.00002 

In [ ]:
!mkdir '/content/drive/MyDrive/iSarcasm/m_298/results'

In [ ]:
shutil.copy("results_h.jsonl", "/content/drive/MyDrive/iSarcasm/m_298/results/results_h.jsonl")
shutil.copy("results_t.jsonl", "/content/drive/MyDrive/iSarcasm/m_298/results/results_t.jsonl")
# shutil.copy("best_model_f.jsonl", "/content/drive/MyDrive/iSarcasm/m_298/results/best_model_f.jsonl")

'/content/drive/MyDrive/iSarcasm/m_298/results/best_model_f.jsonl'

In [ ]:
import operator

In [ ]:
results_h = '/content/drive/MyDrive/iSarcasm/m_298/results/results_h.jsonl'
results_t = '/content/drive/MyDrive/iSarcasm/m_298/results/results_t.jsonl'
best_model_f = "/content/drive/MyDrive/iSarcasm/m_298/results/best_model_f.jsonl"

if os.path.exists(best_model_f):
    os.remove(best_model_f)

results_db_h = pd.read_json(results_h, lines=True)
results_db_t = pd.read_json(results_t, lines=True)

individual_model_val_accs_h = {}
for _, row in results_db_h.iterrows():
    individual_model_val_accs_h[row['model_name']] = row['f1_score']

individual_model_val_accs_h = sorted(individual_model_val_accs_h.items(), key=operator.itemgetter(1))
individual_model_val_accs_h.reverse()
sorted_models = [x[0] for x in individual_model_val_accs_h]



for _, row in results_db_t.iterrows():
    if row['model_name'] == sorted_models[0]:
       row_t = row
       break

results = {'best_model' : f'{sorted_models[0]}'}

results['loss_fn']= row_t['loss_fn']
results['batch_size']=row_t['batch_size']
results['lr']=row_t['lr']
results['epoch']=row_t['epoch']
results['seed']=row_t['seed']
results['f1_sarcastic'] = row_t['f1_sarcastic']
results['f1_score'] = row_t['f1_score']

with open(best_model_f, 'a+') as f:
         f.write(json.dumps(results) + '\n')


In [ ]:
best_model_db_t = pd.read_json(best_model_f, lines=True)
print("best_model_db_t", best_model_db_t)

best_model_db_t           best_model   loss_fn  batch_size       lr  epoch  seed  \
0  36_FL_2e-05_4_298  0.648526          36  0.00002      4   298   

   f1_sarcastic  f1_score  
0      0.648526   0.79141  
